In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/NamO/NamO3.csv"
df = pd.read_csv(file_path)
df.head()

Mounted at /content/drive


,0,4.4493e-05,0.0008458,0.00073085,0.00026002,0.00079209,-0.0001191,-0.0008769,-0.00030082,0.00088563,...,0.0016332,0.0012598,0.0013712,0.0010002.1,0.00051195,0.00034345,0.00087974.1,0.0012298,0.00088875,0.1
0,0,-0.000182,0.000805,0.000616,0.000106,0.000965,-0.000397,-0.000916,-0.000315,0.001260,...,0.001736,0.001297,0.001300,0.000997,0.000221,0.000388,0.001195,0.001208,0.000668,0
1,0,-0.000496,0.000480,0.000219,-0.000206,0.000910,-0.000729,-0.000643,-0.000234,0.001442,...,0.001323,0.000933,0.000738,0.000667,-0.000331,0.000377,0.001344,0.000793,0.000086,0
2,0,-0.000921,-0.000173,-0.000510,-0.000704,0.000607,-0.001151,-0.000147,-0.000112,0.001399,...,0.000444,0.000181,-0.000306,0.000052,-0.001154,0.000263,0.001305,-0.000029,-0.000876,0
3,0,-0.001269,-0.000796,-0.001191,-0.001152,0.000267,-0.001485,0.000374,0.000023,0.001262,...,-0.000456,-0.000572,-0.001321,-0.000551,-0.001881,0.000158,0.001193,-0.000824,-0.001760,0
4,0,-0.001376,-0.001164,-0.001568,-0.001361,-0.000018,-0.001576,0.000660,0.000083,0.000998,...,-0.000997,-0.001018,-0.001883,-0.000864,-0.002181,0.000070,0.000998,-0.001279,-0.002188,0


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader

torch.manual_seed(42)

In [13]:
!pip install torch-geometric
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 39.9 MB/s eta 0:00:00


In [4]:
# Xử lý dữ liệu bị thiếu
df.fillna(method='ffill', inplace=True)

# Chọn một cảm biến để dự báo (ví dụ: cột thứ 2)
data = df.iloc[:, 1].values.reshape(-1, 1)

<ipython-input-4-bd5f48c86a75>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [5]:
# Hàm thêm dữ liệu bị mất
def introduce_missing_data(data, missing_percentage):
    data_missing = data.copy()
    num_missing = int(len(data) * missing_percentage / 100)
    missing_indices = np.random.choice(len(data), num_missing, replace=False)
    data_missing[missing_indices] = np.nan
    return data_missing, missing_indices



In [6]:
# Hàm tạo chuỗi thời gian theo cửa sổ trượt
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [7]:
from sklearn.model_selection import train_test_split

def split_dataset(X, y, train_ratio=0.8):
    return train_test_split(X, y, test_size=1-train_ratio, shuffle=False)


In [17]:
class GCNM(nn.Module):
    def __init__(self, device, num_nodes, dropout, supports, gcn_bool, addaptadj, aptinit,
                 in_dim=1, out_dim=1, residual_channels=32, dilation_channels=32,
                 skip_channels=256, end_channels=512, kernel_size=2, blocks=4, layers=2):
        super(GCNM, self).__init__()

        self.device = device
        self.conv1 = GCNConv(in_dim, residual_channels)
        self.conv2 = GCNConv(residual_channels, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

In [20]:
from torch_geometric.nn import GCNConv
class GCNM_LSTM(nn.Module):
    def __init__(self, device, num_nodes, in_dim, out_dim, lstm_hidden, lstm_layers=1,
                 dropout=0.3, supports=None, gcn_bool=True, addaptadj=True, aptinit=None):
        super(GCNM_LSTM, self).__init__()

        self.device = device
        self.gcnm = GCNM(device, num_nodes, dropout, supports, gcn_bool, addaptadj, aptinit,
                          in_dim=in_dim, out_dim=out_dim,
                          residual_channels=32, dilation_channels=32,
                          skip_channels=256, end_channels=512, kernel_size=2, blocks=4, layers=2)

        # LSTM layer để học đặc trưng thời gian
        self.lstm = nn.LSTM(input_size=out_dim, hidden_size=lstm_hidden, num_layers=lstm_layers,
                            batch_first=True, dropout=dropout, bidirectional=False)

        # Fully Connected Layer để đưa ra dự đoán cuối cùng
        self.fc = nn.Linear(lstm_hidden, out_dim)

    def forward(self, x, x_hist=None):
        """
        x: (B, 8, L, D) hoặc (B, L, D) tùy theo model GCNM hoặc GCNMdynamic
        x_hist: (B, n*tau, L, D) - lịch sử dữ liệu
        """

        # Bước 1: Trích xuất đặc trưng không gian bằng GCNM
        spatial_features = self.gcnm(x, x_hist)  # (B, L, D)

        # Bước 2: LSTM xử lý đặc trưng không gian theo thời gian
        lstm_out, _ = self.lstm(spatial_features)  # (B, L, lstm_hidden)

        # Bước 3: Dự đoán đầu ra
        output = self.fc(lstm_out)  # (B, L, out_dim)

        return output

models = GCNM_LSTM(device='cuda', num_nodes=100, in_dim=1, out_dim=1, lstm_hidden=50, lstm_layers=2, dropout=0.3)


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017Registe

In [30]:
# Chọn tỷ lệ dữ liệu bị mất
missing_percentages = [10, 20, 30]
results = {}

for missing_percentage in missing_percentages:
    print(f"\n*** Xử lý với {missing_percentage}% dữ liệu bị mất ***")

    # Tạo dữ liệu bị mất
    data_missing, missing_indices = introduce_missing_data(data, missing_percentage)

    # Chuẩn hóa dữ liệu
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data_missing)

    # Điền dữ liệu bị thiếu bằng nội suy
    df_filled = pd.DataFrame(data_scaled).interpolate(limit_direction='both').values

    # Điều chỉnh độ dài chuỗi
    if missing_percentage == 30:
        seq_length = 10
    elif missing_percentage == 20:
        seq_length = 20
    else:
        seq_length = 50

    # Tạo tập dữ liệu
    X, y = create_sequences(df_filled, seq_length)

    # Kiểm tra dữ liệu có đủ không
    if X.shape[0] == 0:
        print("Không đủ dữ liệu để huấn luyện. Bỏ qua trường hợp này!")
        continue

    # Chia tập train/test
    X_train, X_test, y_train, y_test = split_dataset(X, y)

    # Huấn luyện mô hình
    num_epochs = 20
    for epoch in range(num_epochs):
        models.train()
        optimizer.zero_grad()
        X_train = X_train.view(X_train.shape[0], seq_length, -1)
        output = models(X_train)

        if edge_index is not None:
            output = models(X_train, edge_index)
        else:
            output = models(X_train, None)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        if epoch % 5 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    # Dự đoán
    X_test = X_test.view(X_test.shape[0], seq_length, -1)
    y_pred = models(X_test).detach().numpy()
    y_pred = scaler.inverse_transform(y_pred)

    # Điền giá trị dự đoán vào dữ liệu gốc
    data_filled_final = data_missing.copy()
    for idx, missing_idx in enumerate(missing_indices[:len(y_pred)]):
        data_filled_final[missing_idx] = y_pred[idx]

    results[missing_percentage] = data_filled_final
    print(f"Dữ liệu bị mất ({missing_percentage}%) đã được dự đoán hoàn tất!")


*** Xử lý với 10% dữ liệu bị mất ***
Không đủ dữ liệu để huấn luyện. Bỏ qua trường hợp này!

*** Xử lý với 20% dữ liệu bị mất ***


ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.

In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ cho từng mức mất dữ liệu
plt.figure(figsize=(15, 8))

for i, missing_percentage in enumerate(missing_percentages):
    if missing_percentage not in results:
        continue  # Bỏ qua nếu không có dữ liệu hợp lệ

    data_filled_final = results[missing_percentage]

    plt.subplot(3, 1, i + 1)
    plt.plot(data, label="Dữ liệu gốc", color='blue', alpha=0.5)
    plt.plot(data_filled_final, label=f"Đã điền ({missing_percentage}%)", color='red', alpha=0.8)

    # Đánh dấu các điểm bị mất ban đầu
    plt.scatter(missing_indices, data[missing_indices], color='black', marker='x', label="Điểm bị mất")

    plt.title(f"Dự đoán cho {missing_percentage}% dữ liệu bị mất")
    plt.legend()
    plt.xlabel("Thời gian")
    plt.ylabel("Giá trị cảm biến")

plt.tight_layout()
plt.show()